In [2]:
#from utils import dump_dataset
import fasttext
import pandas as pd
import nltk
#nltk.download('stopwords') # Added (requested by system)
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

from sklearn.model_selection import train_test_split

stop_words = stopwords.words('french')


df = pd.read_parquet('data/nt_data_fr.gzip')

In [3]:
MAX_WORDS_PER_ARTICLE = 1100
LR = 0.6
EPOCH = 5
DIM = 100
categories = ['sciences','economie','sport','culture', 'sante']
#categories = ['sciences','economie','sport','culture', 'sante', 'suisse', 'people', 'societe', 'insolite', 'politique', 'international', 'tech', 'faits-divers']

In [4]:
def preprocess(sentence, max_words=MAX_WORDS_PER_ARTICLE):
    sentence = sentence.lower()
    
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    word_tokens = tokenizer.tokenize(sentence)
  
    filtered_sentence = [w for w in word_tokens if not w in stop_words][:max_words]
  
    return " ".join(filtered_sentence)

In [5]:
def save_datafile(df, filename):
    with open(filename, 'w') as f:
        for each_text, each_label in zip(df['preproc'], df['category']):
            f.writelines(f'__label__{each_label} {each_text}\n')

In [6]:
df['category'] = df['url'].apply(lambda x :  x.split('/')[3])
df = df[df['category'].isin(categories)]

df['preproc'] = (df['title'] +  " " + df['text']).apply(preprocess)

train, test = train_test_split(df, test_size=0.2)

save_datafile(train, 'data/fasttext.train')
save_datafile(test, 'data/fasttext.valid')

model = fasttext.train_supervised(input="data/fasttext.train", lr=LR,epoch=EPOCH, dim=DIM)
model.test("data/fasttext.valid")

model.save_model("model/categories.ftz")